# Ejercicio 1:

Si le pedimos a Julia la representación en binario de 0.1 (flotante de 64-bits), obtenemos:

In [2]:
bits(0.1)

"0011111110111001100110011001100110011001100110011001100110011010"

Como vimos en clase,los flotantes tienen tres componentes básicos: el signo, el exponente y la mantisa. En éste caso estamos tratando con un flotante de doble precisión, que utiliza 1 bit para el signo, 11 para el exponente y 52 para la mantisa. 

En particular, nos interesan los 11 bits del exponente.


 $01111111011$ ,cuando se codifica el exponente de un punto flotante IEEE 754 de precisión doble, significa $-4$. Éste es el exponente que todos los números entre $0.06125$ (incluido) y $0.125$ (excluido) tienen, y ésto significa que el número es de la forma $m * 2^e$ con $1 ≤ m < 2$ y $e = -4$.
 
Ésta secuencia de bits representa éste exponente porque los exponentes son almacenados en la memoria como enteros sin signo($01111111011$ como un número sin signo es $1019$) con un "bias" de $1023$. El exponende entonces debe ser computado como $1019-1023$, ésto es, $24$.



# Ejercicio 2:

Implementamos la siguiente función recursiva para encontrar el machine epsilon:

In [39]:
function machineepsilon(x,eps=1.0)
    """Encuentra el machine epsilon de un 
    número flotante determinado, recibe como
    argumentos el flotante."""
    if x+ (eps/2) > x #test para ver si el valor de x cambia al sumarle epsilon
        return machineepsilon(x,(eps/2)) #llamada recursiva
        end
    return eps #regresa el valor final de eps, cuando se cumple el caso base x+eps=x
    end


machineepsilon (generic function with 2 methods)

La función inicialmente realiza un test para ver si nuestro epsilon(inicialmente 1) es lo suficientemente grande para alterar el valor de x al ser sumado a él.
Si el test se cumple, la función hace una llamada recursiva de sí misma pero en ésta ocasión es evaluada en x y eps/2. Esto nos indica que, en el peor de los casos,el resultado se obtendrá en un tiempo $O(log_{2}eps)$, por lo que nuestro algoritmo recursivo es eficiente.

Finalmente, cuando el valor de eps es tan pequeño que la suma de éste con x no afecta al valor de x, la función regresa el machine epsilon de x.

A continuación comparamos los machine epsilon de 1.0 y 10.0 obtenidos usando nuestra función con los obtenidos con la funcion eps():

In [38]:
machineepsilon(1.0),eps(1.0)

(2.220446049250313e-16,2.220446049250313e-16)

In [36]:
machineepsilon(10.0),eps(10.0)

(1.7763568394002505e-15,1.7763568394002505e-15)

Por lo tanto, vemos que los machine epsilon obtenidos por nuestra función son correctos y distintos para 1.0 y 10.0

# Ejercicio 3:

Para responder a las preguntas, definimos la función derivada1 en base a la ecuación que aparece en el enunciado del ejercicio:

In [48]:
function derivada1(f,xo,h)
    """Obtiene una aproximación a la derivada de 
    f(x) en x_0. Toma como argumentos a una función,
    a un valor arbitrario de x_0 donde f sea diferenciable,
    y a un h o deltax arbitrario que sea mayor a 0"""
    #@assert h>0 #se asegura que h sea distinta de 0.
    diferencia= (f(xo+h)-f(xo))/h #implementamos la operación que aproximará a la derivada.
    return diferencia #regresa nuestra aproximación de la derivada en x_0
end

derivada1 (generic function with 1 method)

Para poder responder a la pregunta de que tan buena es la aproximación definida por derivada1, estudiaremos al error por aproximación, y luego compararemos valores obtenidos por derivada1 con los valores de las derivadas conocidas para varias funciones.


Con éste fin,primero desarrollaremos la expansión de Taylor de $f(x+h)$ alrededor de x, ésto con el fin de obtener una fórmula matemática que exprese al error de aproximación en términos de $h$:

$$f(x+h)=f(x)+hf'(x)+\frac{h^2f''(\epsilon)}{2}   :\epsilon\in(x,x+h)$$


De donde despejando obtenemos:


$$f'(x)=\frac{f(x+h)-f(x)}{h}-\frac{hf''(\epsilon)}{2}$$


Observamos entonces, que el error es de tiempo $h$, osea, $O(h)$.
Esto nos sugiere que si quisiéramos reducir el error, tendríamos que encontrar un $h$ cuyo valor absoluto sea tan pequeño como sea posible. 

Para estudiar el comportamiento del error, definimos la siguiente función:

In [9]:
f(x)=5x

f (generic function with 1 method)

Sabemos que $f'(x)=5,\forall x$
También sabemos que $f''(x)=0,\forall x$.


Sin perdida de la generalidad podemos decir que, al menos en teoría, el error total de nuestra función al calcular la derivada1 de cualquier funcion lineal debería ser cero, ya que el error por truncamiento es cero.

Ahora probaremos el comportamiento de derivada1 para la función $f(x)=5x$ en $x_{0}=1$, y usaremos 100 valores de $h$ en el intervalo $(eps(),50eps())$, ya que lo que más nos interesa es el comportamiento del error en función de $h$ y cuando $h\rightarrow 0$.

A continuación definimos un vector que nos ayudará a evaluar el comportamiento de derivada1 cuando $h\rightarrow 0$:

In [68]:
i=50eps() #variables para definir un vector de valores en los que evaluaremos derivada1
z=eps()
n=100

vector=linspace(i,z,n) #Construimos un vector de 100-linealmente espaciados números entre eps() y 50eps().

100-element Array{Float64,1}:
 1.11022e-14
 1.09923e-14
 1.08824e-14
 1.07725e-14
 1.06626e-14
 1.05527e-14
 1.04428e-14
 1.03329e-14
 1.0223e-14 
 1.01131e-14
 1.00032e-14
 9.89332e-15
 9.78342e-15
 ⋮          
 1.43095e-15
 1.32105e-15
 1.21115e-15
 1.10125e-15
 9.91351e-16
 8.8145e-16 
 7.71549e-16
 6.61648e-16
 5.51747e-16
 4.41846e-16
 3.31945e-16
 2.22045e-16

Procedemos a evaluar derivada1 en todos los elementos del vector definido en el paso anterior:

In [69]:
for i in vector
    println(i,"  ",derivada1(f,1,i)) #Imprime en la primera columna h, 
    # y en la segunda a los valores de derivada 1 en cada h.
    end  

     

1.1102230246251565e-14  4.96
1.0992329381187661e-14  5.009589879616405
1.0882428516123757e-14  4.978565539983512
1.0772527651059852e-14  5.029356652092442
1.0662626785995948e-14  4.997896508203618
1.0552725920932044e-14  5.049946865037194
1.044282505586814e-14  5.018041237113402
1.0332924190804235e-14  5.071413067071847
1.022302332574033e-14  5.039052215884159
1.0113122460676426e-14  5.093812375249501
1.0003221595612522e-14  4.97219730941704
9.893320730548618e-15  5.0274314214463836
9.783419865484713e-15  4.993122420907841
9.673519000420809e-15  5.049849292835613
9.563618135356903e-15  5.01500938086304
9.453717270292999e-15  5.073309608540926
9.343816405229094e-15  4.942870859337495
9.23391554016519e-15  5.001700267184844
9.124014675101287e-15  4.964601769911504
9.014113810037381e-15  5.025130629509829
8.904212944973478e-15  4.987405541561713
8.794312079909572e-15  5.04973221117062
8.68441121484567e-15  5.011363636363636
8.574510349781764e-15  5.075595082396025
8.46460948471786e-15  5.

Sin embargo observamos que derivada1 no se comporta como esperabamos.

Para cualquier elemento de nuestro vector, derivada1 tendría que haber arrojado $5$, y sin embargo vemos que en algunos casos obtuvimos valores mucho mayores o menores a $5$, aunque bastante cercanos a $5$.

Pero para numeros menores a $1e-16$, derivada1  comenzo a comportarse de forma sorprendente.
Por ejemplo, cuando derivada1 fue evaluada en $eps()$, ésta nos regresó el valor de $4.0$, increíble puesto que ésto implica un error relativo de aproximadamente $8\%$. 

Pero aún más increible es que al ser evaluada en $3.319454699889357e-16$,derivada1 nos arrojó el valor de $2.6756756756756754$, lo que implica un error relativo de más del $50\%$!. Entre más nos acercamos a cero, derivada1 parece alejarse cada vez más del valor esperado.

¿Por qué derivada1 no se comportó como se esperaba?.

La única posible respuesta a ésto es que como el término de la diferencia es dividido por $h$, al hacer $h$ muy pequeño es como si multiplicáramos el numerador de la diferencia por un número muy grande. 
Ésto implica que cualquier error de redondeo (producto de la aritmética de los números de punto flotante) en el numerador se amplificará, y entre más pequeño $h$, éste error aumentará.

Para corroborar ésto,siguiendo la sugerencia del segundo inciso, probemos ahora con un polinomio de grado 3:


In [70]:
g(x)=2x^3+10 

g (generic function with 1 method)

Sabemos analíticamente, que la derivada de ésta función es $g'(x)=6x^2$.


Conocemos que $g'(1)=6$ y que $g''(x)=12x$.
Ahora, escogemos $h\le 50eps()$, y acotamos el error por truncamiento de la siguiente manera:
$$|E_{2}|\le \frac{50eps()12(\epsilon)}{2} : \epsilon\in(1,1+50eps())$$
Lo que implica que:
$$|E_{2}|\le 6.661338147751014e-14 : \epsilon\in(1,1+50eps())$$
Ya que:


In [76]:
(50eps()12(1+50eps()))/2 #Cálculo para determinar cota.

6.661338147751014e-14

Ahora, al igual que en el caso con la función $f(x)=5x$, procedemos a evaluar derivada1 en los elementos de nuestro vector, pero ahora evaluada en $x_0=1$ y $g(x)=2x^3+10$: 

In [78]:
for i in vector
    println(i,"  ",derivada1(g,1,i)) #Imprime en la primera columna h, 
    # y en la segunda a los valores de derivada 1 en cada h.
    end

1.1102230246251565e-14  6.08
1.0992329381187661e-14  6.140787594368496
1.0882428516123757e-14  6.03957131079967
1.0772527651059852e-14  6.101186758276078
1.0662626785995948e-14  5.9974758098443415
1.0552725920932044e-14  6.059936238044633
1.044282505586814e-14  5.953608247422681
1.0332924190804235e-14  6.01693075754287
1.022302332574033e-14  5.907854322071084
1.0113122460676426e-14  5.972055888223553
1.0003221595612522e-14  6.037668161434977
9.893320730548618e-15  6.104738154613466
9.783419865484713e-15  5.991746905089408
9.673519000420809e-15  6.059819151402736
9.563618135356903e-15  5.943714821763603
9.453717270292999e-15  6.012811387900356
9.343816405229094e-15  6.083533365338455
9.23391554016519e-15  6.155938790381346
9.124014675101287e-15  6.035398230088496
9.014113810037381e-15  6.10898233391391
8.904212944973478e-15  5.984886649874055
8.794312079909572e-15  6.059678653404744
8.68441121484567e-15  5.931818181818182
8.574510349781764e-15  6.0078472403871315
8.46460948471786e-15  5

Al igual que en el caso anterior, derivada1 nos arrojó resultados más inexactos entre más pequeña se iba haciendo $h$.
Pero ahora notamos que grandes variaciones respecto al valor esperado se opservan a partir de $1e-15$ y no a partir de $1e-16$ como en el caso anterior. 
Esto se le puede atribuir al hecho de que en ésta ocasión el error por truncamiento era distinto a cero y éste se sumó al error por redondeo de flotantes.

Por ejemplo, vemos derivada1 cuando $h=1.1102230246251565e-14$ nos devolvió $6.08$
Vemos que el error absoluto en éste caso es igual a $0.08$, lo que es mucho mayor al error esperado por nuestra cota.

En ésta ocasión el resultado más alejado del valor esperado fue $8.04$, que implica en éste caso un error relativo de hasta $35\%$.

Por tanto, la función derivada1 funciona correctamente para funciones más o menos arbitrarias, pero entre más nos acercamos a cero la aproximación deja de funcionar como quisiéramos.





# Ejercicio 4:

Definimos ahora la función derivada2:

In [79]:
function derivada2(f,xo,h)
    """Obtiene una aproximación a la derivada de 
    f(x) alrededor de x_0"""
    @assert h>0 #se asegura que h sea mayor a 0
    derivada= (f(xo+h)-f(xo-h))/(2h)
    return derivada
end

derivada2 (generic function with 1 method)

Análogamente al ejercicio anterior, desarrollamos la expansión de Taylor para $f(x+h)$ alrededor de $x$:


$$f(x+h)=f(x)+hf'(x)+\frac{h^2f''(\epsilon)}{2}+\frac{h^2f'''(\epsilon_{1})}{6}:\epsilon_{1}\in(x,x+h)$$

Pero ahora también la desarrollaremos para $f(x-h)$:


$$f(x-h)=f(x)+hf'(x)+\frac{h^2f''(\epsilon)}{2}-\frac{h^2f'''(\epsilon_{2})}{6}:\epsilon_{2}\in(x-h,x)$$


Por lo tanto:


$$f'(x)=\frac{f(x+h)-f(x-h)}{2h}-\frac{h^2[f'''(\epsilon_{1})+f'''(\epsilon_{2})]}{12}$$

Pero por continuidad en el intervalo $(x−h,x+h)$ y usando el teorema del valor intermedio, existe un $\epsilon$ tal que:

$$f'(x)=\frac{f(x+h)-f(x-h)}{2h}-\frac{[h^2f'''(\epsilon_{2})]}{6} :  \epsilon \in (x−h,x+h)$$


Con lo que se observa que el residuo es ahora de tiempo $h^2$ o $O(h^2)$.


A primera impresión podemos pensar que, a $h$ pequeña, el residuo debería disminuir más que el error causado por el redondeo de flotantes en el  numerador de la diferencia, por lo que deberíamos obtenener aproximaciones más exactas.
Vamos a probarlo de forma análoga al ejercicio anterior.

Probamos derivada2 en el intervalo $(eps(),50eps())$, evaluándola en $f(x)=5x$  ,  $ x_0=1$ y todos los $h$ de nuestro vector:

In [82]:
for i in vector
    println(i,"  ",derivada2(f,1,i)) #Imprime en la primera columna h, 
    # y en la segunda a los valores de derivada2 en cada h.
    end

1.1102230246251565e-14  4.96
1.0992329381187661e-14  5.009589879616405
1.0882428516123757e-14  4.978565539983512
1.0772527651059852e-14  5.029356652092442
1.0662626785995948e-14  4.997896508203618
1.0552725920932044e-14  5.007863974495217
1.044282505586814e-14  5.018041237113402
1.0332924190804235e-14  5.028434990232255
1.022302332574033e-14  5.039052215884159
1.0113122460676426e-14  5.049900199600798
1.0003221595612522e-14  4.97219730941704
9.893320730548618e-15  5.0274314214463836
9.783419865484713e-15  4.993122420907841
9.673519000420809e-15  5.003941571991653
9.563618135356903e-15  5.01500938086304
9.453717270292999e-15  5.026334519572955
9.343816405229094e-15  4.942870859337495
9.23391554016519e-15  5.001700267184844
9.124014675101287e-15  4.964601769911504
9.014113810037381e-15  5.025130629509829
8.904212944973478e-15  4.987405541561713
8.794312079909572e-15  4.999234889058914
8.68441121484567e-15  5.011363636363636
8.574510349781764e-15  5.023803295840963
8.46460948471786e-15  5

En éste caso vemos que se comportan casi igual, lo que tiene sentido ya que para cualquier función lineal, el error por truncamiento será de nuevo cero. Aunque si vemos que para los valores más cercanos a cero, la función derivada2 no se aleja tanto del resultado como la función derivada1.

Para la siguiente prueba, definimos y usamos la función $h(x)=10x^4-5$:

In [92]:
h(x)=10x^4-5 

h (generic function with 1 method)

In [93]:
for i in vector
    println(i,"  ",derivada2(h,1,i)) #Imprime en la primera columna h, 
    # y en la segunda a los valores de derivada2 en cada h.
    end

1.1102230246251565e-14  40.0
1.0992329381187661e-14  40.238318710467254
1.0882428516123757e-14  39.991755976916735
1.0772527651059852e-14  40.1524047470331
1.0662626785995948e-14  39.98317206562894
1.0552725920932044e-14  40.231243358129646
1.044282505586814e-14  39.97422680412371
1.0332924190804235e-14  40.14152376817886
1.022302332574033e-14  39.96489688459851
1.0113122460676426e-14  40.22355289421158
1.0003221595612522e-14  39.955156950672645
9.893320730548618e-15  40.12967581047381
9.783419865484713e-15  39.944979367262725
9.673519000420809e-15  40.21516345930906
9.563618135356903e-15  39.93433395872421
9.453717270292999e-15  40.11672597864769
9.343816405229094e-15  39.92318771003361
9.23391554016519e-15  40.20597522467817
9.124014675101287e-15  39.911504424778755
9.014113810037381e-15  40.10251306295099
8.904212944973478e-15  39.8992443324937
8.794312079909572e-15  40.19586840091814
8.68441121484567e-15  39.88636363636363
8.574510349781764e-15  40.086842793617585
8.46460948471786e

Y comparamos la distancia entre el valor esperado ($h'(1)=40$) y el obtenido para derivada1 y derivada 2:

In [95]:
for i in vector
    println(abs(40-derivada2(h,1,i)),"  ",abs(40-derivada1(h,1,i))) #Imprime el error absoluto para cada h
    end

0.0  0.0
0.2383187104672544  0.3999183840032643
0.008244023083264551  0.008244023083264551
0.1524047470331027  0.3997501561524075
0.016827934371058006  0.016827934371058006
0.23124335812964603  0.39957492029755315
0.025773195876290345  0.025773195876290345
0.1415237681788568  0.39939222921641004
0.03510311540149047  0.03510311540149047
0.2235528942115792  0.39920159680638534
0.044843049327354834  0.044843049327354834
0.12967581047381316  0.39900249376558605
0.055020632737274866  0.055020632737274866
0.21516345930906056  0.39879434268490144
0.06566604127579012  0.06566604127579012
0.1167259786476933  0.39857651245552006
0.07681228996639078  0.07681228996639078
0.20597522467816987  0.3983483118775837
0.08849557522124485  0.08849557522124485
0.10251306295099027  0.3981089823339161
0.10075566750629861  0.10075566750629861
0.19586840091813684  0.3978576893649617
0.11363636363636687  0.11363636363636687
0.08684279361758485  0.39759351294794953
0.12718600953894565  0.12718600953894565
0.18469

Como esperabamos, la convergencia de derivada2 en éste caso fue mejor que para derivada1, y en conclusión derivada2 resulta una mejor aproximación a la derivada que derivada1, aunque hasta para derivada2 se sigue observando que para $h$ muy pequeña el error por redondeo sigue afectando de forma tremenda a la aproximación.



### Referencias:
Nocedal, Jorge and Stephen J. wright.
1999.
Numerical Optimization.
Springer-
Verlag New York, Inc.